In [ ]:
import io
import time
from pathlib import Path
import msoffcrypto

CAND_FILE = Path("candidates_sto_ge6.txt")   # updated for 6-char candidates
TARGET_FILE = Path("Mock/Gc_PS7_Mock_test1.docx")
OUT_DECRYPTED = Path("decrypted_output.docx")
PRINT_EVERY = 1000        # progress print frequency
MAX_TRIES = 5000          # increased for more candidates; set to None to try all

if not TARGET_FILE.exists():
    raise FileNotFoundError(f"Target file not found: {TARGET_FILE}")

if not CAND_FILE.exists():
    raise FileNotFoundError(f"Candidates file not found: {CAND_FILE}")

def try_passwords_from_file(candidate_path: Path, target_path: Path,
                            out_decrypted_path: Path,
                            max_tries: int = None, print_every: int = 1000):
    start_time = time.time()
    tried = 0
    with candidate_path.open("r", encoding="utf8", errors="ignore") as f:
        for line in f:
            if max_tries is not None and tried >= max_tries:
                break
            pw = line.rstrip("\n\r")
            if not pw:
                continue
            tried += 1

            # Progress
            if tried % print_every == 0:
                elapsed = time.time() - start_time
                print(f"[{tried}] tried, elapsed {elapsed:.1f}s — last tried: {pw[:80]}")

            try:
                with open(target_path, "rb") as tf:
                    office_file = msoffcrypto.OfficeFile(tf)
                    # load_key will set the password to try
                    office_file.load_key(password=pw)

                    decrypted = io.BytesIO()
                    # Attempt decrypt — this raises on wrong password in many cases
                    office_file.decrypt(decrypted)

                    # Quick sanity check: decrypted buffer non-empty
                    data = decrypted.getvalue()
                    if data and len(data) > 0:
                        # Success
                        out_decrypted_path.write_bytes(data)
                        print(f"SUCCESS! Password found after {tried} tries: {pw}")
                        print(f"Decrypted file written to: {out_decrypted_path.resolve()}")
                        return {"password": pw, "tries": tried, "time_s": time.time() - start_time}
            except Exception as e:
                # wrong password or decrypt error — continue
                # (You can log e for debugging, but avoid flooding)
                # Uncomment next line to debug specific exceptions:
                # print("Exception:", repr(e))
                pass

    print(f"No password found in {tried} attempted guesses.")
    return {"password": None, "tries": tried, "time_s": time.time() - start_time}

# Run it
result = try_passwords_from_file(CAND_FILE, TARGET_FILE, OUT_DECRYPTED, max_tries=MAX_TRIES, print_every=PRINT_EVERY)
print("Result:", result)